In [ ]:
import healpy as hp
import matplotlib.pyplot as plt
import matplotlib as mp
from qubicpack.utilities import Qubic_DataDir
import qubic
import os
import sys
import numpy as np
import glob
from astropy.io import fits

from lmfit.model import load_modelresult

from matplotlib import cm
import matplotlib as mp
from importlib import reload

In [ ]:
def get_maps(file):
    simu = fits.open(file)

    maps_recon = simu['MAPS_RECON'].data
    maps_convo = simu['MAPS_CONVOLVED'].data
    coverage=simu['coverage'].data
    print('coverage shape', coverage.shape)
    diff = maps_recon - maps_convo

    return maps_recon, maps_convo, diff,coverage

def get_maps_many_files(rep_simu, name, verbose=True):
    all_fits = glob.glob(rep_simu + name)
    nfiles = len(all_fits)
    if verbose:
        print('{} files have been found.'.format(nfiles))

    all_maps_recon = []
    all_maps_convo = []
    all_maps_diff = []

    for i, fits in enumerate(all_fits):
        map_recon, map_convo, map_diff,coverage = get_maps(fits)
        if i == 0:
            right_shape = map_recon.shape
        else:
            if map_recon.shape != right_shape:
                raise ValueError('You should take maps with identical shapes.')
        all_maps_recon.append(map_recon)
        all_maps_convo.append(map_convo)
        all_maps_diff.append(map_diff)

    return all_fits, np.asarray(all_maps_recon), \
           np.asarray(all_maps_convo), np.asarray(all_maps_diff), coverage

def read_run(name, nrec = None, repo=None, fixpar=True):
    #repo = os.environ['QUBIC_DATADIR']+'scripts/Spectroimagery_paper/output_paper/nersc/{}/'.format(jobid)
    repo = os.environ['QUBIC_DATADIR']+'scripts/Spectroimagery_paper/output_paper/extended-source/'+name
    repodict = glob.glob(repo+'*.dict')[0]

    d = qubic.qubicdict.qubicDict()
    d.read_from_file(repodict)
    center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
    #if fixpar:
    d['nf_sub']=15
    #print(d['nf_recon'])
    print('d[nf_recon] ',d['nf_recon'])
    #d['nf_recon'] = nrec
    
    _,_,nus_in,_,_,_=qubic.compute_freq(150,d['nf_sub'],0.25)
    
    print("Working with nf_sub = {} and nfrec = {}".format(d['nf_sub'], d['nf_recon']))
    mapsrec={}
    mapsconv={}
    nusfull = {}
    nusfulledge = {}
    for isub in d['nf_recon']:
        _, maprec, mapconv, mapdif,coverage = get_maps_many_files(repo,'*extended*.fits')
        _,inusedge,inus,_,_,_=qubic.compute_freq(150,isub,0.25)
        mapsrec.update({'{}'.format(isub): maprec})
        mapsconv.update({'{}'.format(isub): mapconv})
        nusfull.update({'{}'.format(isub):inus})    
        nusfulledge.update({'{}'.format(isub):inusedge})    
    

    return mapsrec, mapsconv,nusfull,nusfulledge, nus_in, center,d,coverage

In [ ]:
mapsrec, mapsconv, nus, nus_edge, nus_in, center, d, coverage = read_run('simu4/', fixpar=False)
#mapsrec_im, mapsconv_im, nus_im, nus_edge_im, nus_in_im, _, _,_ = read_run('simu3/', fixpar=False)


Reduce dimenssions of maps

In [ ]:
hp.mollview(coverage,rot=center)
mask=coverage>0.1*max(coverage)


In [ ]:
maprec=mapsrec['{}'.format(d['nf_recon'][0])][0]
#print("max rec", np.max(maprec))
mapconv=mapsconv['{}'.format(d['nf_recon'][0])][0]
maxmaps = np.max(mapconv.ravel())
maprec /= maxmaps
mapconv /= maxmaps
nusrec=nus['{}'.format(d['nf_recon'][0])]
nusedge=nus_edge['{}'.format(d['nf_recon'][0])]
fact=d['nf_sub']/d['nf_recon'][0]
for i in range(d['nf_recon'][0]):
    #maprec[i,mask,0]*=fact
    #mapconv[i,mask,0]*=fact
    maprec[i,~mask,0]=hp.UNSEEN
    mapconv[i,~mask,0]=hp.UNSEEN

maprec_im=mapsrec_im['1'][0]
mapconv_im=mapsconv_im['1'][0]
nusrec_im=nus_im['1']
nusedge_im=nus_edge_im['1']

In [ ]:
plt.figure(figsize=(10,10))
plt.rc('font', size=8)
hp.gnomview(maprec[0,:,0],reso=13,rot=center,cmap='jet')

Loading FPSF model... (saved from analysis_psf.Rmd)

In [ ]:
def model_pol(x,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12):
    return a0+a1*x+a2*x**2/2+a3*x**3/3+a4*x**4/4+a5*x**5/5+a6*x**6/6 + \
            a7*x**7/7+a8*x**8/8+a9*x**9/9+a10*x**10/10+a11*x**11/11+a12*x**12/12

result_pol = load_modelresult('polynomial_modelresult_nrec{}.sav'.format(d['nf_recon'][0] \
                                                                         if d['nf_recon'][0] != 5 else \
                                                                        d['nf_recon'][0]+1), 
                              funcdefs={'model_pol': model_pol})
parspol = list(result_pol.best_values.values())

In [ ]:
xlin = np.linspace(-2.,2.)
plt.plot(xlin, model_pol(xlin, *parspol))

Generate same input maps as simulation did...

In [ ]:
inputmap = np.zeros((d['nf_sub'], 12*d['nside']**2,3))
AMP=1e5
center2 = center+np.array([5,5])
pixcen=hp.ang2vec(center2[0],center2[1],lonlat=True)
ip = np.arange(12*d['nside']**2)
vpix = hp.pix2vec(d['nside'],ip)
vecs = np.degrees(np.arccos(np.dot(pixcen,vpix)))
maskcirc = vecs<4

for i in range(-400,400):
    pixcen= hp.ang2pix(d['nside'],center[0],center[1]+i*0.01,lonlat=True)
    inputmap[4,pixcen-30:pixcen+30,0]=AMP

auxmap=np.zeros((12*d['nside']**2))
inputmap[11,maskcirc,0]=AMP
input_editted = np.copy(inputmap)

Generate convolution manually as it is in polyacquisition.py, i.e., considering spectral intensity...

In [ ]:
#sum(width_out) , nus_edge_out

In [ ]:
input_editted = np.copy(inputmap)
#Averaging manually the maps into a band
band = d['filter_nu'] / 1e9
### Input bands
Nfreq_edges, nus_edge, nus, width, _, _ = qubic.compute_freq(band, 
                                                         d['nf_sub'],
                                                         d['filter_relative_bandwidth'])
### Output bands
Nfreq_edges_out, nus_edge_out, nus_out, width_out, _, _ = qubic.compute_freq(band, 
                                                                     d['nf_recon'][0],
                                                                     d['filter_relative_bandwidth'])


# Now averaging maps into reconstruction sub-bands maps
print('Averaging input maps from input sub-bands into reconstruction sub-bands:')
conv_psf = np.zeros((d['nf_recon'][0], 12 * d['nside'] ** 2, 3))
conv_manual = np.zeros((d['nf_recon'][0], 12 * d['nside'] ** 2, 3))
for i in range(d['nf_recon'][0]):
    rel_dist = lambda nui: (nui - nus_out[i])/width_out[i]
    print('doing band {} {} {}'.format(i, nus_edge_out[i], nus_edge_out[i + 1]))
            
    #PSF neigbohur bands
    #option A: conv each map with PSF - average and add it to nu0
    #option B: average - conv with central nu1 and add it to nu0 (this is the chosen one)
    optA=True
    if i == 0:
        inband = (nus > nus_edge_out[i]) & (nus < nus_edge_out[i + 1])
        inbandaux = (nus > nus_edge_out[i + 1]) & (nus < nus_edge_out[i + 2])
        InterBandConv = np.zeros((12 * d['nside']**2, 3))
        if optA:
            #PSF into sub-band
            inmapaux = np.mean([input_editted[jband, :, :] * model_pol(rel_dist(nus[jband]), *parspol) \
                               for jband in np.where(inband)[0]], axis = 0)
            #PSF neigbohur bands
            InterBandConv = np.mean([input_editted[iterband, :, :] * model_pol(rel_dist(nus[iterband]),
                                                             *parspol) for iterband in np.where(inbandaux)[0]],
                   axis=0)
            #hp.gnomview(input_editted[4, :, 0] \
            #            *model_pol(rel_dist(nus[4]),*parspol ), rot=center, reso=12)
        else:
            #PSF into sub-band
            inmapaux = np.mean(input_editted[inband, :, :], axis = 0) * \
                                model_pol(rel_dist(nus[inband]), *parspol),
            #PSF neigbohur bands
            InterBandConv=np.mean(input_editted[inbandaux, :, :], 
                                  axis=0)*model_pol(rel_dist(nus_out[i+1]), *parspol)
        
    elif (i != 0) & (i != d['nf_recon'][0]-1):
        inband = (nus > nus_edge_out[i]) & (nus < nus_edge_out[i + 1])
        InterBandConv = np.zeros((12 * d['nside']**2, 3))
        inband_prev = (nus > nus_edge_out[i - 1]) & (nus < nus_edge_out[i])
        inband_post = (nus > nus_edge_out[i + 1]) & (nus < nus_edge_out[i + 2])
        if optA:
            #PSF into sub-band
            inmapaux = np.mean([input_editted[jband, :, :] * model_pol(rel_dist(nus[jband]), *parspol) \
                               for jband in np.where(inband)[0]], axis = 0)
            #PSF neigbohur bands
            auxmap_prev=np.mean([input_editted[interband_prev, :, :]*model_pol(rel_dist(nus[interband_prev]), 
                                                                            *parspol ) \
                                 for interband_prev in np.where(inband_prev)[0]] ,
                                axis=0)
            auxmap_post=np.mean([input_editted[interband_post, :, :]*model_pol(rel_dist(nus[interband_post]), 
                                                                            *parspol ) \
                                 for interband_post in np.where(inband_post)[0]] ,
                                axis=0)
        else:
            #PSF into sub-band
            inmapaux = np.mean(input_editted[inband, :, :], axis = 0) * \
                                model_pol(rel_dist(nus[inband]), *parspol),
            #PSF neigbohur bands
            auxmap_prev=np.mean(input_editted[inband_prev, :, :], axis=0)*model_pol(rel_dist(nus_out[i-1]), 
                                                                                    *parspol)
            auxmap_post=np.mean(input_editted[inband_post, :, :], axis=0)*model_pol(rel_dist(nus_out[i+1]), 
                                                                                    *parspol)
        InterBandConv=auxmap_prev+auxmap_post
            
        del auxmap_prev, auxmap_post
    elif i == d['nf_recon'][0]-1:
        inband = (nus > nus_edge_out[i]) & (nus < nus_edge_out[i + 1])
        InterBandConv = np.zeros((12 * d['nside']**2, 3))
        inbandaux = (nus > nus_edge_out[i - 1]) & (nus < nus_edge_out[i])
        if optA:
            #PSF into sub-band
            inmapaux = np.mean([input_editted[jband, :, :] * model_pol(rel_dist(nus[jband]), *parspol) \
                               for jband in np.where(inband)[0]], axis = 0)
            #PSF neigbohur bands
            InterBandConv=np.mean([input_editted[iterband, :, :]*model_pol(rel_dist(nus[iterband]),
                                                             *parspol) for iterband in np.where(inbandaux)[0]],
                   axis=0)
        else:
            #PSF into sub-band
            inmapaux = np.mean(input_editted[inband, :, :], axis = 0) * \
                                model_pol(rel_dist(nus[inband]), *parspol),
            #PSF neigbohur bands
            InterBandConv=np.mean(input_editted[inbandaux, :, :], 
                                  axis=0)*model_pol(rel_dist(nus_out[i - 1]), *parspol)
        
    #conv_psf[i, :, :] = np.mean(input_editted[inband, :, :], axis=0) + InterBandConv
    conv_psf[i, :, :] = inmapaux + InterBandConv
    del InterBandConv
    conv_manual[i, :, :] = np.mean(inputmap[inband, :, :], axis=0)
for i in range(d['nf_recon'][0]):
    conv_psf[i,~mask,0] = hp.UNSEEN
    #conv_psf[i,mask,0] *= fact
#
#bands=np.zeros((len(nus_out),2))
#for i in range(len(nus_out)-1):
#    bands[i]= nus_edge_out[i], nus_edge_out[i+1]
#print (bands)
#weights = np.ones_like(nus)
#conv_psf = [np.average(inputmap[(nus > mi)*(nus < ma)], axis=0, 
#                               weights=None) for (mi, ma) in bands   ]

Implement PSF in convolved maps... ok...

plt.figure(figsize=(8,10))
for i in range(d['nf_recon'][0]):
    hp.gnomview(conv_manual[i,:,0], rot=center, reso=12, title='convmanual', sub=(d['nf_recon'][0],4,4*i+1),
               notext=True)
    hp.gnomview(conv_psf[i,:,0], rot=center, reso=12, title='convPSF', sub=(d['nf_recon'][0],4,4*i+2),
               notext=True)
    hp.gnomview(mapconv[i,:,0], rot=center, reso=12, title='conv', sub=(d['nf_recon'][0],4,4*i+3),
               notext=True)
    hp.gnomview(conv_psf[i,:,0]/AMP-mapconv[i,:,0]/AMP, rot=center, reso=12, title='diff=psf-conv', 
                sub=(d['nf_recon'][0],4,4*i+4),
                min=0, max=1e2/AMP,
                notext=True)


In [ ]:
plt.figure(figsize=(8,8))
vmin=-1e4
vmax=1e4
nrec=d['nf_recon'][0]
coln=5
plt.rc('font', size=16)
reso=12
fig,ax=plt.subplots(nrows=nrec,ncols=coln,figsize=(12,16), sharex=True,
                    gridspec_kw={'hspace': 0.01,'wspace': 0.0})
cmap='jet'
ax=ax.ravel()
for i in range(nrec):
    ax[coln*i].cla()
    plt.axes(ax[coln*i])
    hp.gnomview(mapconv[i,:,0],  rot= center,reso=12,notext=True,hold=True,
                max=vmax,
                min=vmin,
                cmap=cmap,
                cbar= True if i == 5 else False,
               title='Convolved' if i == 0 else ' ')
    ax[coln*i+1].cla()
    plt.axes(ax[coln*i+1])
    hp.gnomview(conv_psf[i,:,0],  rot= center,reso=12,notext=True,hold=True,
                max=vmax,
                min=vmin,
                cmap=cmap,
                cbar= True if i == 5 else False,
               title='Convolved (PSF)' if i == 0 else ' ')
    ax[coln*i+2].cla()
    plt.axes(ax[coln*i+2])
    hp.gnomview(maprec[i,:,0],  rot= center,reso=12,notext=True,hold=True,
                max=vmax,
                min=vmin,
                cmap=cmap,
                cbar= True if i == 5 else False,
               title='Recons.' if i == 0 else ' ')
    ax[coln*i+3].cla()
    plt.axes(ax[coln*i+3])
    auxmapconv=(mapconv-maprec)[i,:,0]
    auxmapconv[~mask]=hp.UNSEEN
    hp.gnomview(auxmapconv, rot= center,reso=12,notext=True,hold=True,
                max= vmax,
                min =vmin,
                cmap=cmap,
                cbar= True if i == 5 else False,
               title = 'Dif.Conv' if i == 0 else ' ')
    ax[coln*i+4].cla()
    plt.axes(ax[coln*i+4])
    auxmap=(conv_psf-maprec)[i,:,0]
    auxmap[~mask]=hp.UNSEEN
    hp.gnomview(auxmap, rot= center,reso=12,notext=True,hold=True,
                max= vmax,
                min =vmin,
                cmap=cmap,
                cbar= True if i == 5 else False,
               title = 'Dif.FPSF' if i == 0 else ' ')
    #if i == 0:
    #    ax[coln*i].text(-100,100, 'lala',)

#if d['nf_recon'][0] == 5:
#    plt.text(-2.8,2.80,'{:.0f}GHz'.format(nusrec[0]),fontsize=16)
#    plt.text(-2.8,2.1,'{:.0f}GHz'.format(nusrec[1]),fontsize=16)
#    plt.text(-2.8,1.4,'{:.0f}GHz'.format(nusrec[2]),fontsize=16)
#    plt.text(-2.8,0.7,'{:.0f}GHz'.format(nusrec[3]),fontsize=16)
#    plt.text(-2.8,0,'{:.0f}GHz'.format(nusrec[4]),fontsize=16)
#elif d['nf_recon'][0] == 4: 
#    plt.text(-2.,1.40,'{:.0f}GHz'.format(nusrec[0]),fontsize=16)
#    plt.text(-2.,0.66,'{:.0f}GHz'.format(nusrec[1]),fontsize=16)
#    plt.text(-2.,0,'{:.0f}GHz'.format(nusrec[2]),fontsize=16)
#fig.colorbar(cm.ScalarMappable(), ax=ax[-1])
#plt.savefig('extsource',)#format='pdf')

New consideration. 

Instead computing convolved map using the mean in nu_i and adding the mean of the neighbours sub-band, we multiply all (sub-band centered in nu_i and neig) with FPSF and then averaging

In [ ]:
input_editted = np.copy(inputmap)
#Averaging manually the maps into a band
# Now averaging maps into reconstruction sub-bands maps
print('Averaging input maps from input sub-bands into reconstruction sub-bands:')
conv_psf_new = np.zeros((d['nf_recon'][0], 12 * d['nside'] ** 2, 3))
conv_manual_new = np.zeros((d['nf_recon'][0], 12 * d['nside'] ** 2, 3))
for i in range(d['nf_recon'][0]):
    rel_dist = lambda nui: (nui - nus_out[i])/width_out[i]
    print('doing band {} {} {}'.format(i, nus_edge_out[i], nus_edge_out[i + 1]))
    if i == 0:
        inband = (nus > nus_edge_out[i]) & (nus < nus_edge_out[i + 1])
        inband_post = (nus > nus_edge_out[i + 1]) & (nus < nus_edge_out[i + 2])
        jointband = inband + inband_post
        #Inband
        conv_psf_new[i, :, :] = \
                np.mean([input_editted[inb,:,:]*model_pol(rel_dist(nus[inb]), *parspol) \
                for inb in np.where(jointband)[0]], axis=0)
                    
    elif (i != 0) & (i != d['nf_recon'][0]-1):
        inband_prev = (nus > nus_edge_out[i - 1]) & (nus < nus_edge_out[i])
        inband = (nus > nus_edge_out[i]) & (nus < nus_edge_out[i + 1])
        inband_post = (nus > nus_edge_out[i + 1]) & (nus < nus_edge_out[i + 2])
        jointband = inband_prev + inband + inband_post
        
        conv_psf_new[i, :, :] = \
                np.mean([input_editted[inb,:,:]*model_pol(rel_dist(nus[inb]), *parspol) \
                for inb in np.where(jointband)[0]], axis=0)
        
    elif i == d['nf_recon'][0]-1:
        inband = (nus > nus_edge_out[i]) & (nus < nus_edge_out[i + 1])
        inband_post = (nus > nus_edge_out[i - 1]) & (nus < nus_edge_out[i])
        jointband = inband + inband_post
        
        conv_psf_new[i, :, :] = \
                np.mean([input_editted[inb,:,:]*model_pol(rel_dist(nus[inb]), *parspol) \
                for inb in np.where(jointband)[0]], axis=0)
        
    #print(jointband)
    conv_manual_new[i, :, :] = np.mean(inputmap[inband, :, :], axis=0)
for i in range(d['nf_recon'][0]):
    conv_psf_new[i,~mask,0] = hp.UNSEEN
    #conv_psf[i,mask,0] *= fact
#

In [ ]:
reg_square_conv = np.copy(mapconv[1,:,0])
reg_square_rec = np.copy(maprec[1,:,0])
reg_circular_conv = np.copy(mapconv[3,:,0])
reg_circular_rec = np.copy(mapconv[3,:,0])
leak_0 = np.copy(maprec[0,:,0])
leak_2 = np.copy(maprec[2,:,0])
leak_4 = np.copy(maprec[4,:,0])
mask_square = reg_square_conv < 0.3
mask_circular = reg_circular_conv < 0.3


In [ ]:
mask_circsqua = (mask_circular == 1) & (mask_square == 1)

In [ ]:
#Mask input signal
#mask_circsqau = np.where(mask_circular & mask_square)
reg_square_conv[mask_circsqua] = hp.UNSEEN
leak_0[mask_circsqua] = hp.UNSEEN
leak_2[mask_circsqua] = hp.UNSEEN
plt.figure(figsize = (10,6))
#cmap = copy.copy(mpl.cm.get_cmap("viridis"))
hp.gnomview(reg_square_conv, rot = center, reso = 6, sub = (121))
print("Median", np.median(reg_square_rec[~mask_square]))
hp.gnomview(leak_2, rot = center, reso = 6, sub = (122))

In [ ]:
print(np.median(leak_0[~mask_square]), np.median(leak_2[~mask_circsqua]), np.median(leak_4[~mask_circular]))
print(np.mean([np.median(leak_0[~mask_square]), 
               np.median(leak_2[~mask_circsqua]), np.median(leak_4[~mask_circular])]))

In [ ]:
testmaps = []
cmap = "Reds"#'coolwarm'
for i in range(5):
    testmaps.append(hp.gnomview(maprec[i,:,0], rot = center, cmap = cmap, reso= 8,
                        return_projected_map = True, no_plot = True))
    print("isub {} max = {:.2f}".format(i, np.max(maprec[i,:,0])))
    print("relative with maximum value in rec maps {} ".format(np.max(maprec[i,:,0])/2.8e4))
testmaps = np.array(testmaps)
print(np.shape(testmaps))
testvalues = np.mean(testmaps, axis = 0)
print(np.shape(testvalues))

print("Maximum value = {:.2f} \n Mean value = {:.2e} \n Median value = {:.2e}".format(np.max(testvalues),
                                                                          np.mean(testvalues),
                                                                         np.median(testvalues)))
#print("Median in leak signal {:.2e}".format(np.median(maprec[0,~newmask,0])))

In [ ]:
from matplotlib.colors import Normalize

vmax=1#0.31#np.median(maprec[1,mask,0])
vmin=-1#-vmax

nrec=2
coln=5
plt.rc('font', size=16)
reso=14
fig,ax=plt.subplots(nrows=nrec,ncols=coln,figsize=(18,9), sharex=True,
                    gridspec_kw={'hspace': -0.3, 'wspace': 0.1})

cmap = "coolwarm"#"Reds"
for i in range(5):
    ax[0,i].cla()
    plt.axes(ax[0,i])
    hp.gnomview(mapconv[i,:,0],  rot= center,reso = reso,notext=True,hold=True,
                max=vmax,
                min=vmin,
                cmap=cmap,
                cbar= True if i == 5 else False, return_projected_map = True,
               title=r'{:.1f} GHz'.format(nusrec[i]))
    
    #hp.graticule(dpar = 5, dmer = 5, verbose = False, alpha = 0.6)
    ax[1,i].cla()
    plt.axes(ax[1,i])
    hp.gnomview(maprec[i,:,0], rot= center,reso = reso, notext = True, hold = True,
                max= vmax,
                min =vmin,
                cmap=cmap, title = None,
                cbar= False if i == 4 else False)
hp.graticule(dpar = 5, dmer = 5, verbose = False, alpha = 0.6)
fig.colorbar(cm.ScalarMappable(norm=Normalize(vmin=vmin, vmax=vmax), cmap=cmap), 
             ax=ax[1, 1:4], shrink=0.6, location = "bottom", format = "%.2f",
            ticks = [vmin, vmax], label = "Arbitrary units")



#fig.tight_layout()
#plt.savefig('/home/martin/QUBIC/ArticlesSet_2020/RRPII/extsource_bar_5rec.pdf', format = "pdf", bbox_inches='tight')


In [ ]:
#save maps
lacartes = np.zeros((10, 200, 200))
for i in range(5):
    lacartes[i] = hp.gnomview(mapconv[i,:,0],  rot = center,reso = 9, min = -1, max = 1, 
                              return_projected_map = True, no_plot = True)
    lacartes[5+i] = hp.gnomview(maprec[i,:,0], rot= center,reso = 9, 
                               return_projected_map = True, no_plot = True)

In [ ]:
plt.rc('font', size=16)
reso = 9
fig, ax = plt.subplots(nrows = nrec, ncols = coln, figsize = (18, 12), sharex = True, sharey = True,
                    gridspec_kw = {'hspace': -0.3, 'wspace': 0.1})
ax = ax.ravel()
cmap = "coolwarm"
_cmap_ = "Blues_r"
levels_neg = [-0.2, -0.1,]#0.1, 0.2, 0.3]
levels_pos= [0.2,0.3]

idx = 8
for idx in range(10):
    #plt.axes(ax)
    ax[idx].imshow(lacartes[idx,::-1], vmin = -1, vmax = 1, cmap = cmap )
    CS_n = ax[idx].contour(lacartes[idx, ::-1], levels = levels_neg, cmap = _cmap_, 
                vmin = -1, vmax = 0, linestyles = "solid")
    CS_p = ax[idx].contour(lacartes[idx, ::-1], levels = levels_pos, colors = "k",#cmap = "Reds", 
                vmin = 0, vmax = 1, linestyles = ["solid", "dashed"], alpha = 0.6)
    ax[idx].clabel(CS_p, CS_p.levels, inline=True, fontsize=13)
    ax[idx].clabel(CS_n, CS_n.levels, inline=True, fontsize=13)
    ax[idx].set_yticklabels([])
    ax[idx].set_xticklabels([])    
#fig.colorbar(cm.ScalarMappable(norm=Normalize(vmin=vmin, vmax=vmax), cmap=cmap), 
#             ax=ax[6:9], shrink=0.6, location = "bottom", format = "%.2f",
#            ticks = [vmin, vmax], label = "Arbitrary units")

In [ ]:
mapa_ = hp.gnomview(maprec[1,:,0],  rot= center,reso = 8,notext=True,
        max=vmax, no_plot = True,
        min=vmin,
        cmap=cmap,
        return_projected_map = True, )

In [ ]:
#plt.imshow(mapa_)
mapa_1 = np.copy(maprec[1,:,0])
hp.gnomview(mapa_1, reso = 9, rot = center)
print("media {} median {}".format(np.mean(mapa_1[~mask_square]), np.median(mapa_1[~mask_square])))

In [ ]:
center
x_arr = np.zeros((200))
for i in range(-100,100):
    x_arr[i] = center[1] + (reso / 60) * i 


In [ ]:
from matplotlib.ticker import FormatStrFormatter

X,Y = np.meshgrid(range(200), range(200))

fig = plt.figure(figsize=(18,6))
for ic in range(5):
    ax = fig.add_subplot(1, 5, ic + 1, projection='3d')
    #ax.set_title('cov{} {}-patch {}'.format(bands[ic//2], regions[ic//2][0], config), fontsize=16)
    ax.plot_surface(X, Y, lacartes[ic], cmap=cmap, linewidth=0.2)
    ax.set_zlim(-1, 1)
    if ic < 4: 
        ax.set_zticks([])
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.0e'))
    
fig = plt.figure(figsize=(18,6))
for ic in range(5):
    ax = fig.add_subplot(1, 5, ic + 1, projection='3d')
    #ax.set_title('cov{} {}-patch {}'.format(bands[ic//2], regions[ic//2][0], config), fontsize=16)
    ax.plot_surface(X, Y, lacartes[5 + ic], cmap=cmap, linewidth=0.2)
    ax.set_zlim(0, 1)
    if ic < 4: 
        ax.set_zticks([])
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.0e'))
#fig.colorbar(cm.ScalarMappable(norm=Normalize(vmin = -2e4, vmax = 2e4), cmap = cmap), 
#             ax=ax[1:4], shrink=0.6, location = "bottom", format = "%.1e",
#            ticks = [vmin, vmax], label = "Arbitrary units")

plt.show()


#### Trying to make a 3d plot

In [ ]:
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.plot_surface(X, Y, lacartes[5]+lacartes[6], cmap=cmap, linewidth=0.2)
